In [6]:
!git clone https://github.com/KL0224/RetrievalPerson -b pipeline

Cloning into 'RetrievalPerson'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 53 (delta 9), reused 50 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 9.32 MiB | 30.58 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [7]:
!pip install open_clip_torch
!pip install torchreid
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchreid: filename=torchreid-0.2.5-py3-none-any.whl size=144324 sha256=2989ed43c8dd3d5bc022b091919e74008b3382445e17b0e87fe8b9068b839445
  Stored in directory: /root/.cache/pip/wheels/5c/86/ff/80a1b78a90df470cbb12c075bf189ad33f1a41a881cf9e9a09
Successfully built torchreid
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.7 MB/s eta 0:00:00a 0:00:01


In [8]:
%cd /kaggle/working/RetrievalPerson

/kaggle/working/RetrievalPerson


In [9]:
import os

VIDEO_FOLDER = '../../input/dataset-person/videos'
OUTPUT_FOLDER = '/kaggle/working/outputs'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_FOLDER, 'frames'), exist_ok=True)

In [10]:
import numpy as np
import glob
import os
import PIL.Image as Image
import cv2

from config import *
from tracking.tracklet import TrackletManager
from tracking.detector_tracker import run_tracking
from sampling.sampler import sample_best_per_window
from models.reid import ReIDModel
from models.clip_model import CLIPModel

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.12/dist-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
E0000 00:00:1767361104.439552      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767361104.487858      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767361104.876117      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767361104.876155      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767361104.876157      55 computation_placer.cc:177] computation placer already regis

In [11]:
def save_image_webp(img_bgr, path: str, quality: int = 80, resize_factor: float = 0.5):
    if resize_factor != 1.0:
        img_bgr = cv2.resize(img_bgr, (0, 0), fx=resize_factor, fy=resize_factor)
    img_pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    os.makedirs(os.path.dirname(path), exist_ok=True)
    img_pil.save(path, format="WEBP", quality=quality)

In [12]:
manager = TrackletManager()

In [13]:
# tracking
seqs = sorted(glob.glob(f'{VIDEO_FOLDER}/seq_*'))

for seq in seqs[:len(seqs)//2]:
    seq_name = os.path.basename(seq)
    seq_id = int(seq_name.split('_')[-1])
    print(f'Processing sequence {seq_name}')
    
    os.makedirs(os.path.join(OUTPUT_FOLDER, 'frames', seq_name), exist_ok=True)
    cameras = sorted(glob.glob(f'{seq}/camera_*'))
    for cam_id, video_path in enumerate([cameras[0]]):
        cam_id += 1
        camera_name = "_".join(os.path.basename(video_path).split('_')[:2])
        camera_frame_folder = os.path.join(OUTPUT_FOLDER, 'frames', seq_name, camera_name)
        os.makedirs(camera_frame_folder, exist_ok=True)
        print(f'  Processing camera {cam_id}')
        for frame_id, frame, boxes, ids, confs in run_tracking(video_path, vid_stride=1, confidence=CONFIDENCE_THRESHOLD, save=True, 
                                                           project_name=f'{OUTPUT_FOLDER}/annotated_videos', 
                                                           name=seq_name,
                                                           show=False):
            
            frame_save_path = os.path.join(camera_frame_folder, f'frame_{frame_id:06d}.webp')
            save_image_webp(frame, frame_save_path)
            for box, tid, conf in zip(boxes, ids, confs):
                gid = seq_id*SEQ_ID_OFFSET + cam_id * CAMERA_ID_OFFSET + tid
                x, y, w, h = map(int, box)
                x1, y1 = x, y
                x2, y2 = x + w, y + h
                
                crop = frame[y1:y2, x1:x2]

                t = manager.get(gid, seq_id, cam_id)
                t.add_frame(frame_id, box, conf, crop)

Processing sequence seq_000
  Processing camera 1
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 233ms
Prepared 1 package in 71ms
Installed 1 package in 2ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



KeyboardInterrupt: 

In [14]:
# Sampling + Embeddings
reid_model = ReIDModel()

for t in manager.all():
    sampled = sample_best_per_window(t.frames)
    candidates = []
    if len(sampled) <= 3:
        candidates = sampled
    else:
        candidates = sampled[:1] + sampled[-1:] + sampled[len(sampled)//2:len(sampled)//2+1]

    imgs = [f.image for f in candidates]

    reid_feats = reid_model.extract(imgs).mean(axis=0)
    t.reid_embeddings.append(reid_feats)

/usr/local/lib/python3.12/dist-packages/torchreid/reid/utils/tools.py:43: UserWarning: No file found at "osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"
  warnings.warn('No file found at "{}"'.format(fpath))
Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 179MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352


In [15]:
del reid_model

In [16]:
# Sampling + Embeddings
clip_model = CLIPModel()

for t in manager.all():
    sampled = sample_best_per_window(t.frames)
    candidates = []
    if len(sampled) <= 3:
        candidates = sampled
    else:
        candidates = sampled[:1] + sampled[-1:] + sampled[len(sampled)//2:len(sampled)//2+1]

    imgs = [f.image for f in candidates]

    clip_feats = np.array([clip_model.encode_image(img) for img in imgs]).mean(axis=0)
    t.clip_embeddings.append(clip_feats)


open_clip_pytorch_model.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

In [18]:
del clip_model

NameError: name 'clip_model' is not defined

In [20]:
tracklets = manager.all()

# save into metadata.txt
with open(f"{OUTPUT_FOLDER}/metadata.txt", "a") as f:
    for t in tracklets:
        for frame in t.frames:
            f.write(f"{t.sequence_id} {t.camera_id} {frame.frame_id} {t.global_id} {frame.bbox[0]} {frame.bbox[1]} {frame.bbox[2]} {frame.bbox[3]}\n")

# save into features.txt
with open(f"{OUTPUT_FOLDER}/features.txt", "w") as f:
    for t in tracklets:
        f.write(f"{t.sequence_id} {t.camera_id} {t.global_id} {t.reid_embeddings[0].tolist()} {t.clip_embeddings[0].tolist()}\n")

In [ ]:
!zip -r results.zip /kaggle/working/outputs
from IPython.display import FileLink
FileLink(r'results.zip')